# Data

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [2]:
df = pd.read_csv('/Users/ningyuhan/Desktop/combined_dataframe.csv')

## Tokenize Setence

In [3]:
def standardize_text(text):
    if pd.isna(text):
        return text  
    # Remove non-ASCII characters directly
    text = ''.join([char for char in text if ord(char) < 128])
    text = text.strip().lower()  
    # Remove all non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

df['Standardized_Sentence'] = df['Sentence'].apply(standardize_text)

def standardize_tag(tag):
    if pd.isna(tag):
        return tag
    # Remove all spaces and extra characters
    tag = tag.replace(' ', '')
    # Ensure consistency in formatting
    tag = ','.join(tag.split(','))
    return tag

df['Tag'] = df['Tag'].apply(standardize_tag)

## Correct Typos

In [4]:
df['Tag'] = df['Tag'].apply(
    lambda x: x.replace('4h,30]', '4h,03]') if '4h,30]' in x else x
)

def correct_typo(tag):
    parts = tag.strip('[]').split(',')
    if len(parts) > 3 and len(parts[3]) == 1:
        parts[3] = '0' + parts[3]  
    return '[' + ','.join(parts) + ']'

df['Tag'] = df['Tag'].apply(correct_typo)

# Three Sentence after "4h" Instances

In [5]:
df['Tag_Component'] = df['Tag'].apply(lambda x: x.split(',')[2] if len(x.split(',')) > 2 else None)

# Set the focus category here
focus_category = '4h'
df['Is_Focus_Category'] = df['Tag_Component'] == focus_category

## If Meet Consective "4h" Sentences, Only Keep the Last One

In [6]:
def get_relevant_rows_after(df):
    relevant_rows = []
    in_consecutive_4h = False  # Flag to track if we're in a series of consecutive '4h' tagged rows

    for i in range(len(df) - 1):  # Adjusted to avoid index out of range
        current_is_4h = df.iloc[i]['Is_Focus_Category']
        next_is_4h = df.iloc[i + 1]['Is_Focus_Category']

        # If the current row is '4h' and the next row is not, or it's the last '4h' in a series
        if current_is_4h and not next_is_4h and in_consecutive_4h:
            # Reset the flag since this is the last '4h' in a series
            in_consecutive_4h = False
            # Add the last '4h' row in the series
            relevant_rows.append(df.iloc[i])
            # Add the next three non-'4h' rows
            for j in range(1, 4):
                if i+j >= len(df):  # Check if the index goes beyond the DataFrame
                    break
                if not df.iloc[i+j]['Is_Focus_Category']:
                    relevant_rows.append(df.iloc[i+j])
                else:
                    break  # Stop if another '4h' tagged row is encountered

        # If the current row is '4h' and it's the first in a series or standalone
        elif current_is_4h and not in_consecutive_4h:
            if not next_is_4h:  # If the next row is not '4h', treat it as a standalone '4h' row
                relevant_rows.append(df.iloc[i])
                # Add the next three non-'4h' rows
                for j in range(1, 4):
                    if i+j >= len(df):
                        break
                    if not df.iloc[i+j]['Is_Focus_Category']:
                        relevant_rows.append(df.iloc[i+j])
                    else:
                        break
            else:  # If the next row is also '4h', set the flag and skip this row
                in_consecutive_4h = True

    return pd.DataFrame(relevant_rows).sort_index()

In [7]:
df_after = get_relevant_rows_after(df)
df_after

,Tag,Sentence,Standardized_Sentence,Tag_Component,Is_Focus_Category
12,"[c:13,13,4h,01]",Dodson got mad at Ralph,dodson got mad at ralph,4h,True
13,"[c:14,12,2b,01]",The discussion on compensation was most unfort...,the discussion on compensation was most unfort...,2b,False
14,"[c:15,11,4e2,01]",compensation was most unfortunate.,compensation was most unfortunate,4e2,False
15,"[c:16,19,4b,01]","Earl wanted full compensation, arguing that th...",earl wanted full compensation arguing that the...,4b,False
49,"[c:50,02,4h,09]","Everybody talked about the ""armed camp"" atmosp...",everybody talked about the armed camp atmosphe...,4h,True
...,...,...,...,...,...
4838,"[c:90,25,4a,06]",and Lyle simply says that he wants to make sur...,and lyle simply says that he wants to make sur...,4a,False
4849,"[c:101,25,4h,04]","but Lyle says, ""Yes, he was fired.Ó",but lyle says yes he was fired,4h,True
4850,"[c:102,22,1b,03]",Laverne says he's holding a meeting to save th...,laverne says hes holding a meeting to save the...,1b,False
4851,"[c:103,22,4e3,03]",that includes getting financing to the tune of...,that includes getting financing to the tune of...,4e3,False


# Topics after "4h" Sentences

In [8]:
topic_codes = {
    '01': 'Routine Board Functions',
    '02': 'Scope of Board Issues',
    '03': 'Financial Status of Firm',
    '04': 'Management Personnel Issues',
    '05': 'Marketing and Sales',
    '06': 'Employee Stock Ownership Plan (ESOP) Financial',
    '07': 'ESOP Participation',
    '08': 'Employee Benefits',
    '09': 'Plant Production and Manufacturing Process Issues',
    '10': 'The Union and Contract Issues',
    '11': 'Supervision',
    '12': 'hog procurement',
    '13': 'Political Relations with the Community'
}

## Topic Counts

In [9]:
# Extract the fourth component (Topic) from the 'Tag' field
df_after['Topic_Component'] = df_after['Tag'].apply(
    lambda x: x.split(',')[3].strip(']').strip() if len(x.split(',')) > 3 else None
)

# Filter the DataFrame for rows where 'Is_Focus_Category' is True
df_focus_true = df_after[df_after['Is_Focus_Category']]

# Count the occurrences of each unique 'Topic_Component' in these rows
topic_component_counts = df_after['Topic_Component'].value_counts()

# Translate the topic components using the topic_codes dictionary
translated_topic_counts = {topic_codes.get(key, "Unknown"): value for key, value in topic_component_counts.items()}

# Display the translated counts
translated_topic_counts

# Calculate the total count for percentage calculation
total_count = topic_component_counts.sum()

# Translate the topic components and calculate the percentage
translated_topic_counts = {
  topic_codes.get(key, "Unknown"): {
    'Count': value, 
    'Percentage': (value / total_count) * 100
  }
  for key, value in topic_component_counts.items() 
}

for topic, data in translated_topic_counts.items():
  percentage = round(data['Percentage'], 2)
  data['Percentage'] = f"{percentage}%"

for topic, data in translated_topic_counts.items():
  print(f"{topic}: {data['Count']} times ({data['Percentage']})")

Financial Status of Firm: 196 times (20.0%)
Management Personnel Issues: 181 times (18.47%)
The Union and Contract Issues: 135 times (13.78%)
Routine Board Functions: 99 times (10.1%)
Plant Production and Manufacturing Process Issues: 83 times (8.47%)
ESOP Participation: 78 times (7.96%)
Marketing and Sales: 62 times (6.33%)
Employee Benefits: 50 times (5.1%)
Employee Stock Ownership Plan (ESOP) Financial: 47 times (4.8%)
Supervision: 24 times (2.45%)
hog procurement: 16 times (1.63%)
Political Relations with the Community: 6 times (0.61%)
Scope of Board Issues: 3 times (0.31%)


## Topic Sequence after "4h" Sentences

In [10]:
sequences_following_focus_topic = []

for index in range(len(df_after)):
    if df_after.iloc[index]['Is_Focus_Category']:
        topic_sequence = []
        # Look ahead for the next three non-'4h' rows
        for j in range(1, 4):  # Start from the next row
            if index + j >= len(df_after):  # Check if the index goes beyond the DataFrame
                break
            if not df_after.iloc[index + j]['Is_Focus_Category']:
                topic_sequence.append(df_after.iloc[index + j]['Topic_Component'])
        sequences_following_focus_topic.append(tuple(topic_sequence))  # No need to reverse

sequence_counts = Counter(sequences_following_focus_topic)
top10_sequences = sequence_counts.most_common(10)

# Consolidate duplicates
consolidated_counts = {}
for seq, count in top10_sequences:
    consolidated_seq = tuple(set(seq))  # Remove duplicate topics within the sequence
    if consolidated_seq not in consolidated_counts:
        consolidated_counts[consolidated_seq] = 0
    consolidated_counts[consolidated_seq] += count

# Translate codes
translated_counts = {}
for seq, count in consolidated_counts.items():
    translated_seq = tuple(topic_codes.get(code, 'Unknown') for code in seq)  
    if translated_seq not in translated_counts:
        translated_counts[translated_seq] = 0
    translated_counts[translated_seq] += count

# Get total count
total_count = sum(count for seq, count in top10_sequences)

# Print results with percentage
print("Top Sequences Following Focus Topic (Translated):")
for translated_seq, count in translated_counts.items():
    percentage = round(count / total_count * 100, 2)
    print(f"{translated_seq}: {count} times ({percentage}%)")

Top Sequences Following Focus Topic (Translated):
('Financial Status of Firm',): 42 times (29.58%)
('Management Personnel Issues',): 34 times (23.94%)
('Routine Board Functions',): 23 times (16.2%)
('The Union and Contract Issues',): 14 times (9.86%)
('Plant Production and Manufacturing Process Issues',): 12 times (8.45%)
('ESOP Participation',): 10 times (7.04%)
('Employee Stock Ownership Plan (ESOP) Financial',): 7 times (4.93%)


In [11]:
top10_sequences

[(('03', '03', '03'), 30),
 (('04', '04', '04'), 25),
 (('01', '01', '01'), 16),
 (('10', '10', '10'), 14),
 (('09', '09', '09'), 12),
 (('03', '03'), 12),
 (('07', '07', '07'), 10),
 (('04', '04'), 9),
 (('01', '01'), 7),
 (('06', '06', '06'), 7)]

# Verbal Contribution after "4h" Sentences

In [12]:
verbal_contribution_codes = {
    '1': 'Initiation activity',
    '1a': 'Motion making',
    '1b': 'Information giving',
    '1c': 'Information seeking',
    '1d': 'Making specific suggestions for action',
    '2': 'Support behavior',
    '2a': 'Motion seconding',
    '2b': 'Making statements in support of another person\'s argument',
    '3': 'System maintenance',
    '3a': 'Tension management',
    '3b': 'Direction of traffic',
    '3c': 'Collective spirit and solidarity moves',
    '4': 'Board discussion, debate, argumentation',
    '4a': 'Personal defensiveness',
    '4b': 'Personal gains',
    '4c': 'Agreeing reluctantly',
    '4d': 'Sensible,nonpersonal arguments',
    '4e1': 'Management',
    '4e2': 'Union',
    '4e3': 'People (workers) as distinct from union or the union leadership',
    '4f': 'Attempts to propose new board topics',
    '4g': 'Corporate interests',
    '4h': 'Disagreements, conflicts, attacks',
    '4i': 'Stonewalling',
    '5': 'Unclassified verbal behaviors',
}

In [13]:
def get_following_rows(df):
    following_rows = []

    for i in range(len(df)):
        if df.iloc[i]['Is_Focus_Category']:
            sequence = []
            # Look ahead for the next three non-'4h' rows
            for j in range(1, 4):
                if i+j >= len(df):
                    break
                if not df.iloc[i+j]['Is_Focus_Category']:
                    sequence.append(df.iloc[i+j]['Tag_Component'])
            following_rows.append(tuple(sequence))

    return following_rows

# Analyze sequences following the '4h' instances
sequences_following_focus = get_following_rows(df_after)

# Count the frequency of each sequence
sequence_counts = Counter(sequences_following_focus)
top10_sequences = sequence_counts.most_common(10)

# Format the results
formatted_results = []
for sequence, count in top10_sequences:
    sequence_str = ' -> '.join(sequence)
    sentence = f"[{sequence_str}] follows {focus_category} {count} times."
    formatted_results.append(sentence)

# Print formatted results
for result in formatted_results:
    print(result)

[1c -> 1b -> 4d] follows 4h 5 times.
[1a -> 2a -> 1b] follows 4h 4 times.
[4d -> 4d -> 1b] follows 4h 4 times.
[1b -> 4d -> 1c] follows 4h 4 times.
[1c -> 1b -> 1b] follows 4h 4 times.
[4e2 -> 1c] follows 4h 3 times.
[4d -> 1c -> 4d] follows 4h 3 times.
[4d -> 4d -> 4d] follows 4h 3 times.
[4a -> 4a] follows 4h 3 times.
[4a -> 4d] follows 4h 3 times.


In [14]:
print(df_after['Is_Focus_Category'].sum())

270


## Category 3 after "4h" ?

In [15]:
sequences_with_3a_3b_3c = [seq for seq in sequences_following_focus if any(code in seq for code in ["3a", "3b", "3c"])]
print(len(sequences_with_3a_3b_3c))
sequences_with_3a_3b_3c

61


[('1c', '4e1', '3a'),
 ('3a', '4a', '1c'),
 ('3a', '4d'),
 ('4d', '4d', '3b'),
 ('4d', '3b', '1c'),
 ('4i', '3a'),
 ('3a', '4a', '3a'),
 ('3a', '4a'),
 ('4a', '3a'),
 ('3a', '1d', '4c'),
 ('05', '3a', '1c'),
 ('1b', '1b', '3a'),
 ('1b', '3a', '3a'),
 ('1b', '4d', '3a'),
 ('4i', '3b', '1d'),
 ('4g', '3a', '1c'),
 ('4a', '3b'),
 ('3b', '1b', '1b'),
 ('3a', '4e2', '4e1'),
 ('3a', '4i', '1d'),
 ('3a', '4c', '1b'),
 ('1b', '1c', '3a'),
 ('05', '3b'),
 ('05', '05', '3b'),
 ('3a', '4e2', '1c'),
 ('1c', '4d', '3b'),
 ('3a', '1b', '4d'),
 ('3b', '1b'),
 ('4g', '3b', '3b'),
 ('4d', '1b', '3a'),
 ('3a', '4e2'),
 ('4a', '3b', '1b'),
 ('3a', '3a', '4g'),
 ('3b', '1c', '4d'),
 ('1b', '3b', '2b'),
 ('3b', '4e1'),
 ('4d', '3a'),
 ('3a', '3b', '3b'),
 ('4d', '3b', '4d'),
 ('4i', '3a', '4e1'),
 ('3b', '4c', '1b'),
 ('4b', '05', '3b'),
 ('1a', '2a', '3b'),
 ('4d', '3b', '4c'),
 ('3a', '4d', '1d'),
 ('4e2', '4d', '3a'),
 ('3b', '1c', '4d'),
 ('3a', '1c', '3b'),
 ('3b', '4f', '1c'),
 ('1b', '3b'),
 ('3b', 

# People Who Speak after "4h"

In [16]:
data = {
    'Name': ['Chuck Swisher', 'Emmet “Mac” MacGuire', 'Herb Epstein', 'John Lambert', 'Wally Rath, regional sales managers, & Robert Cray & John DeGroat', 'Bob Soleday, Mowry, Bloomfield,(beginning Jan. 1985)', 'Ivan Pihl', 'Art Frye', 'Jack Thomas', 'Harold Rath', 'Ralph Helstein', 'Tove Hammer', 'Len Dodson', 'Sid Oberman', 'Bob Fulton', 'Walter Cunningham', 'Dick Clarke', 'Phyllis Walters', 'Earl Murray', 'Glen Bass', 'Clark Towne', 'LaVerne Patrie', 'Peter Bruskern', 'Bob Kavangh', 'Lyle Taylor', 'Chuck Mueller', 'Gene Redmond & other union officials', 'Jim Miller', 'Ron Peterson, William Scogland, Charles McCarthy, Tom Mandler, Wes Hall, Ravel', 'Berthold', 'Rudnick', 'Gerjerts, Bill Wait', 'Jack Curtis', 'Bruce Wilson', 'John Stevens, Greg Kohn, Lewis Rudel', 'Potential Business deal people', 'ESOP consultants (Chris Meek & Warner Woodworth, W.F. Whyte)', 'ESOP trustees, Larry Wrede, Jim Anderson, Cox', 'Wayne Wright', 'Insurance people & other consultants', 'Leroy Grittman, Tobias, Mary Frost', 'Swisher & Cohrt, Law firm (Steve Weidner)'],
    'Id Number': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']
}

person_df = pd.DataFrame(data)

In [17]:
df_after['Person_Component'] = df_after['Tag'].apply(
    lambda x: x.split(',')[1].strip(']').strip() if len(x.split(',')) > 3 else None
)

In [18]:
sequences_following_focus_person = []

# Iterate through the DataFrame
for index in range(len(df_after) - 1):  
    # Check if the current row is tagged with '4h'
    if df_after.iloc[index]['Is_Focus_Category']:
        person_sequence = []
        # Look ahead to the next three rows, provided they exist and are not tagged as '4h'
        for j in range(1, 4):
            if index + j < len(df_after) and not df_after.iloc[index + j]['Is_Focus_Category']:
                person_sequence.append(df_after.iloc[index + j]['Person_Component'])
        # Append the sequence, if any, to the list
        if person_sequence:
            sequences_following_focus_person.append(tuple(person_sequence))

sequences_following_focus_person

[('12', '11', '19'),
 ('01', '08'),
 ('15', '08', '15'),
 ('15', '04'),
 ('04', '04', '04'),
 ('02', '04', '12'),
 ('20', '04', '12'),
 ('01', '02'),
 ('02', '02', '15'),
 ('19', '02', '20'),
 ('11', '02', '04'),
 ('20', '02', '20'),
 ('11', '02', '01'),
 ('12', '01', '01'),
 ('12', '02'),
 ('02', '01'),
 ('01', '02', '11'),
 ('01', '37'),
 ('12', '04'),
 ('04', '01'),
 ('01', '11', '37'),
 ('25', '25', '25'),
 ('08', '02', '08'),
 ('25', '12'),
 ('12', '08', '19'),
 ('11', '01', '01'),
 ('09', '08', '02'),
 ('20', '08', '20'),
 ('37', '08', '11'),
 ('08', '02'),
 ('02', '26', '11'),
 ('01', '25'),
 ('25', '25', '01'),
 ('12', '02', '11'),
 ('12', '02', '17'),
 ('11', '01', '20'),
 ('15', '18', '02'),
 ('10', '01', '01'),
 ('04', '20'),
 ('20', '09', '11'),
 ('04', '20', '04'),
 ('02', '02'),
 ('20', '12', '01'),
 ('04', '15', '15'),
 ('13', '01', '12'),
 ('14', '09', '08'),
 ('09', '03', '03'),
 ('03', '01', '01'),
 ('04', '12'),
 ('09', '14', '01'),
 ('26', '03', '08'),
 ('26', '26',

In [19]:
# Flatten the list of tuples to count each id
flattened_ids = [id for sequence in sequences_following_focus_person for id in sequence]

# Count each id
id_counts = Counter(flattened_ids)

# Convert the counter object to a DataFrame
df_id_counts = pd.DataFrame(id_counts.items(), columns=['ID', 'Count'])

# Merge the count DataFrame with the translation DataFrame
merged_df = pd.merge(df_id_counts, person_df, left_on='ID', right_on='Id Number', how='left')

# Select only the necessary columns
final_df = merged_df[['ID', 'Count', 'Name']]

final_df

,ID,Count,Name
0,12,72,Tove Hammer
1,11,46,Ralph Helstein
2,19,16,Earl Murray
3,01,89,Chuck Swisher
4,08,34,Art Frye
5,15,43,Bob Fulton
6,04,27,John Lambert
7,02,29,Emmet “Mac” MacGuire
8,20,36,Glen Bass
9,37,3,"John Stevens, Greg Kohn, Lewis Rudel"
